In [ ]:
import integrator
import numpy as np
import numba
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import os

## Earth Moon System Example

In [ ]:
rmoon = 385000e3
m1 = 7.35e22
m2 = 5.97e24
G = 6.67430e-11
yr2sec = 3.154e7
vmoon = np.sqrt(G*m2/rmoon)

part_pos = np.array([[0,-rmoon*m2/(m1+m2),0],[0,rmoon*m1/(m1+m2),0]],dtype=np.float64)
part_vel = np.array([[vmoon,0,0],[-m1/m2*vmoon,0,0]],dtype=np.float64)
part_mass = np.array([m1,m2],dtype=np.float64)
dt = yr2sec/1e4
num_dt = int(1e6)
t = np.linspace(0,num_dt*dt,num_dt+1)
save_pos = np.zeros((num_dt+1,2,3))
save_vel = np.zeros((num_dt+1,2,3))

integrator.leapfrog_int_nb(part_pos,part_vel,part_mass,dt,num_dt,save_pos,save_vel)

In [ ]:
plt.figure()
plt.plot(save_pos[:,0,0],save_pos[:,0,1])
plt.plot(save_pos[:,1,0],save_pos[:,1,1])
plt.ylabel('y position')
plt.xlabel('x position')
plt.legend(['%.2e kg particle'%(part_mass[0]),
            '%.2e kg particle'%(part_mass[1])])
plt.title('1 Year (12 orbits) of Earth-Moon System')
plt.show()

In [ ]:
# Check the timing - roughly 5 micro seconds per step for 2 body
%timeit integrator.leapfrog_int_nb(part_pos,part_vel,part_mass,dt,num_dt,save_pos,save_vel)

## e=0.9 Example

In [ ]:
# Next we want to test the orbital properties of an e = 0.9 Kepplerian orbit
e = 0.9
rp = 1.9
vt = 0.005927
period = 243
m1 = 1e2
m2 = 1e7
rcm = m1*rp/(m1+m2)
vcm = vt/rp*rcm

# For plotting the correct orbit
a = rp*(1-e)/(1-e**2)

part_pos = np.array([[rp,0,0],[0,0,0]],dtype=np.float64)
part_vel = np.array([[0,vt-vcm,0],[0,-vcm,0]],dtype=np.float64)
part_mass = np.array([m1,m2],dtype=np.float64)
dt = period/2010.6
num_dt = int(period/dt*200)
t = np.linspace(0,num_dt*dt,num_dt+1)
save_pos = np.zeros((num_dt+1,2,3))
save_vel = np.zeros((num_dt+1,2,3))

integrator.leapfrog_int_nb(part_pos,part_vel,part_mass,dt,num_dt,save_pos,save_vel)

In [ ]:
plt.figure(figsize=(5,3),dpi=200)

# Plot the orbits subsampled
steps_per_period = int(np.ceil(period/dt))
fudge = 0
skip = 10
for i in range(20):
    plt.plot(save_pos[skip*i*steps_per_period:(skip*i+1)*steps_per_period,0,0]-
             save_pos[skip*i*steps_per_period:(skip*i+1)*steps_per_period,1,0],
             save_pos[skip*i*steps_per_period:(skip*i+1)*steps_per_period,0,1]-
             save_pos[skip*i*steps_per_period:(skip*i+1)*steps_per_period,1,1],c='k')
    
# Plot the analytic solution
theta = np.linspace(0,2*np.pi,2000)
r = -a*(1-e**2)/(1+e*np.cos(theta))
x_an = r*np.cos(theta)
y_an = r*np.sin(theta)
plt.plot(x_an,y_an,c='b')

plt.axvline(0,c='k')
plt.axhline(0,c='k')
plt.ylabel('y position')
plt.xlabel('x position')
plt.title('e=0.9 Leapfrog Integration')
plt.show()

## 3 Body Problem

In [ ]:
part_pos = np.array([[2,0,0],[0,0,0],[0,2,0]],dtype=np.float64)
part_vel = np.array([[0,1e-4,0],[0,0,0],[0,-1e-4,0]],dtype=np.float64)
part_mass = np.array([100,100,10],dtype=np.float64)
dt = 0.01
num_dt = int(1e8)
t = np.linspace(0,num_dt*dt,num_dt+1)
save_pos = np.zeros((num_dt+1,3,3))
save_vel = np.zeros((num_dt+1,3,3))

integrator.leapfrog_int_nb(part_pos,part_vel,part_mass,dt,num_dt,save_pos,save_vel)

In [ ]:
plt.figure()
cut = int(8e7)
plt.plot(save_pos[:cut:10000,0,0]-save_pos[:cut:10000,0,0],save_pos[:cut:10000,0,1]-save_pos[:cut:10000,0,1])
plt.plot(save_pos[:cut:10000,1,0]-save_pos[:cut:10000,0,0],save_pos[:cut:10000,1,1]-save_pos[:cut:10000,0,1])
plt.plot(save_pos[:cut:10000,2,0]-save_pos[:cut:10000,0,0],save_pos[:cut:10000,2,1]-save_pos[:cut:10000,0,1])
# plt.xlim((-15,20))
# plt.ylim((-5,50))
plt.ylabel('y position')
plt.xlabel('x position')
plt.title('3 Bodies')
plt.show()

In [ ]:
time_step = int(2e5)
length = int(8e7)
sub_samp = int(1e5)
images = []

for i in range(int(length//time_step)):
    if i%10 == 0:
        print(i,time_step*i)
    fig = plt.figure(figsize=(5,5),dpi=100)
    plt_cut = time_step*i
    plt.plot(save_pos[:plt_cut:sub_samp,0,0],save_pos[:plt_cut:sub_samp,0,1])
    plt.plot(save_pos[:plt_cut:sub_samp,1,0],save_pos[:plt_cut:sub_samp,1,1])
    plt.plot(save_pos[:plt_cut:sub_samp,2,0],save_pos[:plt_cut:sub_samp,2,1])
    plt.ylabel('y position')
    plt.xlabel('x position')
    fig.savefig('temp_%i.png'%(i))
    plt.close()
    im = Image.open('temp_%i.png'%(i))
    images.append(im)

print('making the gif')
images[0].save('3b.gif',save_all=True,append_images=images[1:],
              optimize=False, duration=40, loop=0)

print('cleaning up')
for i in range(int(length//time_step)):
    os.remove('temp_%i.png'%(i))

In [ ]:
from galpy.potential import NFWPotential

@numba.njit()
def calc_neg_grad_nfw(rho_0,r_scale,pos_nfw,pos):
	""" Calculate the negative gradient of the nfw potential at a specific
		position.

		Parameters:
			rho_0 (float): the density normalization parameter for the NFW
			r_scale (float): The scale radius for the NFW profile
			pos_nfw (np.array): The 3D position of the NFW
			pos (np.array): The 3D position to calculate the gradient at

	"""
	norm = 4 * np.pi * G * rho_0 * r_scale**3
	r2 = np.sum(np.square(pos_nfw-pos))
	r = np.sqrt(r2)
	r_hat = (pos_nfw-pos)/r

	return norm*(1/r2 * np.log(1+r/r_scale) - 1/(r_scale+r)*1/r) * r_hat

In [ ]:
r_scale = 2
rho_0 = 1
nfw = NFWPotential(a=r_scale,amp=rho_0*G*4*np.pi*r_scale**3)

In [ ]:
nfw.Rforce(1,0)

In [ ]:
pos_nfw = np.zeros(3,dtype=np.float64)
pos = np.zeros(3,dtype=np.float64)
pos[0] = 1
calc_neg_grad_nfw(rho_0,r_scale,pos_nfw,pos)-nfw.Rforce(1,0)

In [ ]:
G